# stackoverflow

https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix

In [1]:
import numpy as np

In [2]:
arr= np.array([[ 1,  0,  0,  1,  1,  0],
   [ 1,  0,  0,  0,  0,  0],
   [ 1,  1,  1,  1,  0,  0],])

In [3]:
arr

array([[1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0]])

array([[1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0]])

In [4]:
(arr[:, None, :] != arr).sum(2)

# array([[0, 2, 3],
#        [2, 0, 3],
#        [3, 3, 0]])
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

In [5]:
compute_HammingDistance(arr)

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

# setup

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata

BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images
sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [7]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# hash

In [8]:
def hamming(x1, x2): 
    x1,x2 = (bin(int(x, 16)) for x in (x1, x2))
    return sum(o1!=o2 for o1,o2 in zip(x1,x2))
n = 5
hdists = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(i, n):
        x1 = df.image_phash[i]
        x2 = df.image_phash[j]
        hdists[i,j]=hdists[j,i]=hamming(x1, x2)
hdists

100%|██████████| 5/5 [00:00<00:00, 7373.95it/s]


array([[ 0., 42., 40., 30., 32.],
       [42.,  0., 28., 28., 34.],
       [40., 28.,  0., 34., 44.],
       [30., 28., 34.,  0., 34.],
       [32., 34., 44., 34.,  0.]])

100%|██████████| 5/5 [00:00<00:00, 7373.95it/s]


array([[ 0., 42., 40., 30., 32.],
       [42.,  0., 28., 28., 34.],
       [40., 28.,  0., 34., 44.],
       [30., 28., 34.,  0., 34.],
       [32., 34., 44., 34.,  0.]])

In [9]:
hfeats = df[:5].image_phash.apply(lambda x: bin(int(x, 16))).values

In [10]:
hfeats

array(['0b1001010010010111010011111001001101111101010011000010010000110011',
       '0b1010111100111111100101000110000011000010100000111000111100001111',
       '0b1011100101001100101100000000111011010011111001010000111101111000',
       '0b1000010100010100111111000101100011101010111111101010001010000011',
       '0b1010011011110011000110011111100100100100101011010111000010001100'],
      dtype=object)

array(['0b1001010010010111010011111001001101111101010011000010010000110011',
       '0b1010111100111111100101000110000011000010100000111000111100001111',
       '0b1011100101001100101100000000111011010011111001010000111101111000',
       '0b1000010100010100111111000101100011101010111111101010001010000011',
       '0b1010011011110011000110011111100100100100101011010111000010001100'],
      dtype=object)

In [11]:
[eval(o) for o in hfeats[0][2:]]

[1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1]

[1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1]

In [12]:
X = np.array([[eval(o) for o in x[2:]] for x in hfeats])
X

array([[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

array([[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

In [13]:
X.dtype

dtype('int64')

dtype('int64')

In [14]:
X = X.astype(np.short)

In [15]:
compute_HammingDistance(X)

array([[ 0, 42, 40, 30, 32],
       [42,  0, 28, 28, 34],
       [40, 28,  0, 34, 44],
       [30, 28, 34,  0, 34],
       [32, 34, 44, 34,  0]])

array([[ 0, 42, 40, 30, 32],
       [42,  0, 28, 28, 34],
       [40, 28,  0, 34, 44],
       [30, 28, 34,  0, 34],
       [32, 34, 44, 34,  0]])

# all data

In [16]:
hfeats = df.image_phash.apply(lambda x: bin(int(x, 16))).values

In [17]:
%%time
X = np.array([[eval(o) for o in x[2:]] for x in hfeats])


CPU times: user 8.54 s, sys: 16.8 ms, total: 8.55 s
Wall time: 8.55 s
CPU times: user 8.54 s, sys: 16.8 ms, total: 8.55 s
Wall time: 8.55 s


In [18]:
X = X.astype(np.short)

In [ ]:
%%time
hdist = compute_HammingDistance(X)